# Python Primer for Bioinformatics

Full name: Usama Khan

Grade achieved: 98%


## Deadline and submission

Complete this notebook with your code, compress it and submit it as a single .zip or .gz file through the QMPlus page by the following deadline:

    Friday Nov 3rd 2023, 16:00


## Preliminary reading

Please read the PDF file ```BIO723P_Assignment_Background.pdf``` distributed with this assigment carefully before you start coding.

# Question 1: Primers and melting temperature

### Question 1.a: Reading a DNA sequence from a file

Write a python function called ```readDNAsequence``` that takes as its argument the name of a file.  When passed the name of a FASTA file, the function should read the file, discard the header and return the sequence as a string. Your code should raise ```BadSequenceException``` (defined below) if the sequence part of the file contains characters that are not one of the letters A, C, T, G, U. All U nucleotides should be replaced by T in the returned string (for simplicity, we will be working with T only throughout the rest of this assignment).

In [1]:
# Run this cell to define the exception
class BadSequenceException(Exception):
    pass

In [2]:
# Your code here
# Your code here
# Defining our sequence which we are calling readDNAsequence.
# The argument will be a FASTA filename.
def readDNAsequence(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

        # Assuming that the first line is the header-skip
        sequence_lines = lines[1:]

        # Join the sequences, remove newlines, and replace U with T (case-insensitive)
        content = ''.join(sequence_lines).replace('\n', '').replace('U', 'T').replace('u', 't').upper()

        # Check for invalid DNA characters
        # If no invalid characters were found, return the sequence.
        # The 'sequence' is a string that contains the DNA sequence read from the file,
        # with all occurrences of 'U' replaced with 'T', all line endings removed,
        # and all characters converted to uppercase.
        # This sequence is then available to be used by the code that called this function.
        if any(char not in 'ACGT' for char in content):
            raise BadSequenceException(f"Invalid DNA character found: {char}")

        return content

In [3]:
# Test code
import os
with open("test9876345.fas", "wt") as _OUTF:
    _OUTF.write("> test\n")
    _OUTF.write("ACTG\n")
_seq=readDNAsequence("test9876345.fas")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
os.remove("test9876345.fas")
print("OK")

OK


### Question 1.b: Computing the complement of a sequence

Write a function called ```complement``` that takes a string containing a DNA sequence as its only  parameter and returns the complement of the sequence in a string. The function should raise ```BadSequenceException``` if the argument sequence contains anything else than the four characters A, C, T, G. Do not reverse the string; for the avoidance of doubt, if the input string starts with A then the complement string should start with T.


In [4]:
# Your code here
# Writing a function called complement, that 
# takes a DNA sequence
def complement(sequence):
    # This dictionary defines the complement for each nucleotide
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    
    # Initialise an empty string to build the complement sequence
    complement_sequence = ''
    
    # Iterate over each nucleotide in the input sequence
    for nucleotide in sequence:
        # If the nucleotide is not one of the valid characters, raise an exception
        if nucleotide not in complement_dict:
            raise BadSequenceException(f"Invalid DNA character found: {nucleotide}")
        # Append the complement of the nucleotide to the complement sequence
        complement_sequence += complement_dict[nucleotide]
    
    # Return the complement sequence
    return complement_sequence

In [5]:
# Test code
_seq=complement("ACGTTTCG")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print("OK")

OK


### Question 1.c: Extracting primers

Write a function called ```primer``` that takes three parameters: a DNA sequence called ```sequence```, an integer ```length``` that is 20 by default and a Boolean value ```forward``` that is ```True``` by default. When ```forward``` is ```True``` (or is not passed), the function should return a Forward primer for the sequence passed as ```sequence```; when it is ```False```, it should return a  Reverse primer. The length of the primer is specified by ```length```; if this is not passed, a primer of length 20 should be returned.  Refer to the Background document for a definition of primers and how to compute them (for the avoidance of doubt, if the sequence string ends with a C, then the reverse primer string should start with a G). If the sequence is shorter than ```length``` nucleotides, your code should raise a ```BadSequenceException```.


In [6]:
# Your code here
# Let us begin to define a function called primer.
# primer takes Sequence, Length, and Forward as inputs/arguments.
# Length is set to 20 by default. Forward is True by default.
# Define the primer function which uses the complement function from 1b
def primer(sequence, length=20, forward=True):
    # Check if the sequence is shorter than the requested primer length.
    # If so, we raise an exception.
    if len(sequence) < length:
        raise BadSequenceException(f"Sequence length is shorter than the requested primer length of {length} nucleotides.")
    
    if forward:
        return sequence[:length]
    else:
        reverse_sequence = sequence[-length:][::-1]
        return complement(reverse_sequence)

In [7]:
# Test code
_seq=primer(sequence="AAAAATTTTTCCCCCGGGGGAAAAA", length= 20, forward=False)
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print("OK")

OK


### Question 1.d: Computing the melting temperature

Write a function called ```meltingTemp``` that takes a string representing a primer as its argument. The function should return the melting temperature of the primer in degrees Celsius according to the equation given in the Background document. If the sequence contains characters other than A, C, T, G, the function should raise a ```BadSequenceException```.

In [8]:
# Your code here
# Write a function called meltingTemp that takes a string representing a primer as its argument.
def meltingTemp(primer):
    primer = primer.upper()  # Convert to uppercase
    invalid_chars = []  # List to store any invalid characters found

    # Check for invalid characters
    for nucleotide in primer:
        if nucleotide not in 'ATGC':
            invalid_chars.append(nucleotide)

    # If any invalid characters were found, raise an exception listing all of them
    if invalid_chars:
        raise BadSequenceException(f"Invalid characters found: {', '.join(invalid_chars)}")

    # Count the occurrences of each nucleotide type
    count_A = primer.count('A')
    count_T = primer.count('T')
    count_G = primer.count('G')
    count_C = primer.count('C')

    # Calculate melting temperature using the given formula
    Tm = 4 * (count_G + count_C) + 2 * (count_A + count_T)
    return Tm

In [9]:
# Test code
_temp=meltingTemp("AAAAATTTTTCCCCCGGGGG")
assert ((type(_temp) is type(0.0)) or
        (type(_temp) is type(0))), "Return value is not a number: %r" % _temp
print("OK")

OK


### Question 1.e: Putting it all together

Write a function called ```sequencePCRtemp``` that takes a string containing the name of a FASTA file as its argument. The function should return the average melting temperature of the two primers of the sequence as a ```float```.

In [10]:
# Your code here
# Write a function called sequencePCRtemp
# FASTA file as argument
def sequencePCRtemp(fasta_filename):
    # Read the sequence from the FASTA file
    sequence = readDNAsequence(fasta_filename)
    # Extracting fwd primer
    forward_primer = primer(sequence, length=20, forward=True)
    # Extracting reverse primer
    reverse_primer = primer(sequence, length=20, forward=False)
    # Compute melting temperatures for primers
    forward_temp = meltingTemp(forward_primer)
    reverse_temp = meltingTemp(reverse_primer)
    # Calculate average melting temperature of primers
    average_temp = (forward_temp + reverse_temp) / 2
    return average_temp

In [11]:
# Test code
import os
with open("test9876346.fas", "wt") as _OUTF:
    _OUTF.write("> test\n")
    _OUTF.write("AAAAACCCCCTTTTTGGGGGAAAAA\n")
_temp=sequencePCRtemp("test9876346.fas")
assert type(_temp) is type(0.0), "Return value is not a float: %r" % _temp
os.remove("test9876346.fas")
print("OK")

OK


# Question 2: Translation and reading frames

### Question 2.a: Reading frames
Write a function ```translate``` that takes a string containing a DNA sequence as its input and outputs a Python dictionary containing the translation of the sequence in all possible reading frames. The keys of the dictionary should be ```f1```, ```f2```, ```f3``` for the three forward frames and ```r1```, ```r2``` and ```r3``` for the reverse reading frames; the value of each key should be the translation of the sequence in the corresponding frame.
For simplicity and ease of debugging, **do not complement the sequence** when computing the reverse reading frames; just reverse it. Use an asterisk (```*```) to represent stop codons. Always translate the entire sequence.

In [12]:
# Your code here
from typing import Dict

# Creating a codon table since the process of translation
# will require a reference point for our DNA sequence
CODON_TABLE: Dict[str, str] = {
    "AAA": "K", "AAC": "N", "AAG": "K", "AAT": "N",
    "ACA": "T", "ACC": "T", "ACG": "T", "ACT": "T",
    "AGA": "R", "AGC": "S", "AGG": "R", "AGT": "S",
    "ATA": "I", "ATC": "I", "ATG": "M", "ATT": "I",
    "CAA": "Q", "CAC": "H", "CAG": "Q", "CAT": "H",
    "CCA": "P", "CCC": "P", "CCG": "P", "CCT": "P",
    "CGA": "R", "CGC": "R", "CGG": "R", "CGT": "R",
    "CTA": "L", "CTC": "L", "CTG": "L", "CTT": "L",
    "GAA": "E", "GAC": "D", "GAG": "E", "GAT": "D",
    "GCA": "A", "GCC": "A", "GCG": "A", "GCT": "A",
    "GGA": "G", "GGC": "G", "GGG": "G", "GGT": "G",
    "GTA": "V", "GTC": "V", "GTG": "V", "GTT": "V",
    "TAA": "*", "TAG": "*", "TGA": "*",
    "TAC": "Y", "TAT": "Y",
    "TCA": "S", "TCC": "S", "TCG": "S", "TCT": "S",
    "TGC": "C", "TGG": "W", "TGT": "C",
    "TTA": "L", "TTC": "F", "TTG": "L", "TTT": "F"
}

def translate(dna_sequence: str) -> Dict[str, str]:
    # Despite instruction that we will only recieve
    #upper case sequence, inserting function just 
    # for completeness.
    dna_sequence = dna_sequence.upper()

    # Generate the three forward and three reverse reading frames 
    # directly from the DNA sequence
    frames = {
        'f1': dna_sequence,
        'f2': dna_sequence[1:],  # Remove first nucleotide
        'f3': dna_sequence[2:],  # Remove first two nucleotide
        'r1': dna_sequence[::-1],  # Reverse sequence
        'r2': dna_sequence[-2::-1],  # Reverse removing the first nucleotide
        'r3': dna_sequence[-3::-1],  # Reverse removing the first two nucleotide
    }

    # Translate reading frames by cross referencing our codon table
    translated_frames = {}
    for frame_label, frame_sequence in frames.items():
        protein_sequence = ""
        for i in range(0, len(frame_sequence), 3):
            codon = frame_sequence[i:i+3]
            if len(codon) == 3:  
                amino_acid = CODON_TABLE.get(codon, '?') 
                protein_sequence += amino_acid
        translated_frames[frame_label] = protein_sequence

    return translated_frames

In [13]:
# Test code
_seqdic=translate("ACTGACTGACTGACTGACTGACTG")
assert type(_seqdic)==type(dict()), "Return value is not a dictionary: %r" % _seqdic
assert set(_seqdic.keys())==set(['f1', 'f2', 'f3', 'r1', 'r2', 'r3']), \
    "Output dictionary has incorrect/missing keys: %r"  % _seqdic.keys()
assert type(_seqdic['f1'])==type(""), \
    "Output dictionary values should be strings, not %r" % type(_seqdic['f1'])
print("OK")

OK


### Question 2.b: Locating an ORF

Write a function called ```openReadingFrame``` that takes a string containing an aminoacid sequence as its argument and returns a string containing the aminoacids between the first Methionine (included) and the first STOP codon that follows it (excluded). Assume the stop codon is represented by an asterisk (```*```) as would be returned by ```translate``` above. If either the Methionine or the STOP codon are missing, your function should return an empty string.

In [14]:
# Your code here
# Write a function called openReadingFrame (ORF)
#Function should take aminoacid sequence as argument
def openReadingFrame(amino_acid_sequence: str) -> str:
    # Find the index of the first 'M' in the sequence
    start_index = amino_acid_sequence.find('M')
    
    # No 'M'? return an empty string
    if start_index == -1:
        return ""
    
    # Find the index of the first '*' after the 'M'
    stop_index = amino_acid_sequence.find('*', start_index)
    
    # If there is no '*', return an empty string
    if stop_index == -1:
        return ""
    
    # Return the sequence from the 'M' to just before the '*'
    return amino_acid_sequence[start_index:stop_index]

In [15]:
# Test code
_seq=openReadingFrame("AMCAPP*L")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print("OK")

OK


### Question 2.c: Translating a sequence

Write a function called ```candidateProtein``` that takes a string containing a DNA sequence as its input and outputs the string of aminoacids corresponding to the longest ORF, as extracted by ```openReadingFrame``` above.

In [16]:
#Write a function called candidateProtein
#Argument of function will be a dna_sequence
def candidateProtein(dna_sequence: str) -> str:

    # Calling on our previous translate function to get our reading frames
    translated_frames = translate(dna_sequence)
    
    # Initialise an empty string which will hold our longest protein
    longest_protein = ""

    # Loop through the translated frames and get the openreadingframe (ORF) for each frame
    for frame_sequence in translated_frames.values():
        orf = openReadingFrame(frame_sequence)
        
        # Compare the length of the current ORF to the longest protein
        #update longest protein if necessary
        if len(orf) > len(longest_protein):
            longest_protein = orf

    # Return the longest protein
    return longest_protein

In [17]:
# Test code
_seq=candidateProtein("ATGACTGCTGGGTAG")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print("OK")

OK


### Question 2.d: Writing a FASTA file

Write a function called ```writeFASTA``` that takes three string arguments called, in the order, ```sequence```, ```description``` and ```filename```. Argument ```sequence``` should contain an aminoacid sequence. Argument ```description``` should contain a description (eg name of protein, organism, etc). Argument ```filename``` should contain a file name. Your code should create the file with the name requested, write to  it the description as a FASTA header (i.e. starting with the character ```>```) and write the sequence to the file. Long sequences should be formatted over several lines. The function should not return any value.

In [18]:
# Your code here
# Write a function called writeFASTA
# Takes three string arguments (sequence, description, and filename)
def writeFASTA(sequence: str, description: str, filename: str):
    # Open the file for writing
    with open(filename, "w") as f:
        # Write the description line with a '>' prefix
        f.write(">%s\n" % description)
        # Loop through the sequence in chunks of 20 characters
        for i in range(0, len(sequence), 20):
            # Write a chunk of 20 characters of the sequence to the file
            f.write(sequence[i:i + 20] + "\n")

In [19]:
# Test code
import os
import os.path
_rv=writeFASTA(sequence="TESTTESTTESTTESTTEST",
              description="test sequence",
              filename="test9876347.fas")
assert type(_rv) is type(None), "Function should not return anything; it returns %r" % _rv
_fe=os.path.isfile("test9876347.fas")
assert _fe, "Cannot find output file - has it been created?"
os.remove("test9876347.fas")
print("OK")

OK


### Question 2.e: Putting it all together


Write a function called ```maximalORF``` that takes as its argument string ```inputfile``` containing the name of an input file, string ```outputfile``` with the name of an output file and string  ```proteinname``` with a description of a candidate protein. The function should read a DNA sequence from the input file and write the candidate protein corresponding to the longest ORF to the output file, in FASTA format. The string supplied in ```proteinname``` should provide the header of the FASTA file. The function should not return any value.

In [20]:
# Your code here
# Write a function called maximalORF (open reading frame)
def maximalORF(inputfile: str, outputfile: str, proteinname: str):

    # Read the DNA sequence from the input file.
    with open(inputfile, "r") as f:
        dna_sequence = f.read()

    # Translate the DNA sequence into reading frames.
    translated_frames = translate(dna_sequence)

    # Identify the longest ORF in each reading frame.
    longest_ORFs = {}
    for frame_label, frame_sequence in translated_frames.items():
        longest_ORFs[frame_label] = openReadingFrame(frame_sequence)

    # compare lengths of longest ORFs -selecting the longest one.
    longest_ORF = ""
    for frame_label, orf in longest_ORFs.items():
        if len(orf) > len(longest_ORF):
            longest_ORF = orf

    # Writing longest ORF to the output file in FASTA format.
    writeFASTA(longest_ORF, proteinname, outputfile)

In [21]:
# Test code
import os
import os.path
with open("test9876348.fas", "wt") as _OUTF:
    _OUTF.write("> test\n")
    _OUTF.write("ATGACTGCTGGGTAG\n")
_rv=maximalORF(inputfile="test9876348.fas", outputfile="test9876349.fas",
               proteinname="test protein")
assert type(_rv) is type(None), "Function should not return anything; it returns %r" % _rv
_fe=os.path.isfile("test9876349.fas")
assert _fe, "Cannot find output file - has it been created?"
os.remove("test9876348.fas")
os.remove("test9876349.fas")
print("OK")

OK


# Question 3: Classification of aminoacids

### Question 3.a: Reading an aminoacid sequence

Write a function called ```readAAsequence``` that takes as its argument the name of a file.  When passed the name of a FASTA file containing an aminoacid sequence, the function should read the file, discard the header and return the sequence as a string.

In [22]:
# Your code here
def readAAsequence(filename):
    # Initialise an empty string to store the sequence.
    sequence = ""
    try:
        # Open file.
        with open(filename, 'r') as f:
            # Iterate through each line.
            for line in f:
                # If the line starts with ">"?- we skip it.
                if line.startswith(">"):
                    continue
                sequence += line.strip()
        return sequence
    except FileNotFoundError:
        return ""

In [23]:
# Test code
import os
with open("test9876350.fas", "wt") as _OUTF:
    _OUTF.write("> test\n")
    _OUTF.write("TESTTESTTESTTESTTEST\n")
_seq=readAAsequence("test9876350.fas")
os.remove("test9876350.fas")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print ("OK")

OK


### Question 3.b: Aminoacid usage statistics

Write a function called ```AAtypes``` that takes as its argument a string containing a sequence of aminoacids. The function should compute, for each protein, the fraction of aminoacids that are polar, small and hydrophobic and return that as a list. For instance, if the chain of a protein contains 35% polar residues, 15% small and 50% hydrophobic residues, the function should return ```[0.35, 0.15, 0.5]``` (the order is important). Refer to the Venn diagram in the Assessment Background file for the classification of aminoacids (other Venn diagrams may differ).  Note that some aminoacids belong to more than one category, so that the sum of these three numbers can be larger than 1.

In [24]:
# Your code here
# Write a function called AAtypes
#argument? amino_acid_sequence

def AAtypes(amino_acid_sequence):
    #Classification of amino acids transcribed from venn diagram
    Classification_of_Amino_Acids = {
        'Polar': ['R', 'K', 'H', 'W', 'Y', 'E', 'N', 'S', 'T', 'D', 'Q', 'C'],
        'Small': ['G', 'N', 'S', 'T', 'P', 'A', 'D', 'V', 'C'],
        'Hydrophobic': ['F', 'W', 'Y', 'H', 'K', 'T', 'C', 'M', 'A', 'I', 'L', 'V']
    }
    
    Length_of_amino_acid_sequence = len(amino_acid_sequence)
    #Counting as the names of the functions below suggest
    def Number_of_amino_acids_which_are_polar(sequence):
        count = 0
        for amino_acid in sequence:
            if amino_acid in Classification_of_Amino_Acids['Polar']:
                count += 1
        return count
    
    def Number_of_amino_acids_which_are_Small(sequence):
        count = 0
        for amino_acid in sequence:
            if amino_acid in Classification_of_Amino_Acids['Small']:
                count += 1
        return count
    
    def Number_of_amino_acids_which_are_Hydrophobic(sequence):
        count = 0
        for amino_acid in sequence:
            if amino_acid in Classification_of_Amino_Acids['Hydrophobic']:
                count += 1
        return count
    #Dividing our count of amino acid type by amino acid length gives us proportions
    def Percentage_of_amino_acid_sequence_Polar(sequence):
        return Number_of_amino_acids_which_are_polar(sequence) / Length_of_amino_acid_sequence

    def Percentage_of_amino_acid_sequence_Small(sequence):
        return Number_of_amino_acids_which_are_Small(sequence) / Length_of_amino_acid_sequence
    
    def Percentage_of_amino_acid_sequence_Hydrophobic(sequence):
        return Number_of_amino_acids_which_are_Hydrophobic(sequence) / Length_of_amino_acid_sequence

    result = [
        Percentage_of_amino_acid_sequence_Polar(amino_acid_sequence),
        Percentage_of_amino_acid_sequence_Small(amino_acid_sequence),
        Percentage_of_amino_acid_sequence_Hydrophobic(amino_acid_sequence)
    ]
    return result

In [25]:
# Test code
_aatypes=AAtypes("TESTTESTTESTTESTTEST")
assert type(_aatypes) is type([]), "Return value is not a list: %r" % _aatypes
assert len(_aatypes)==3, "Returned list should contain 3 values"
print("OK")

OK


### Question 3.c: Processing multiple sequences
Write a function called ```AAtypetable``` that takes a list of strings called ```filelist``` and a separate string called ```outputfile```. Each element of ```filelist``` represents the name of a FASTA file containing an aminoacid sequence. For each filename, the function should load the sequence, compute the fraction of aminoacids that are polar, small and/or hydrophobic, and finally output this to the text file specified by ```outputfile``` in TSV (tab-separated values) tabular format. The table should contain, on each line, the name of the input file and then the percentages of polar, small and hydrophobic residues (in that order). Include a comment at the beginning of the file that starts with a ```#``` and specifies the content of each column, as follows (numbers and file names shown here are random):
```
# Filename     Polar     Small     Hydro
Pxxx.fasta     0.52      0.22      0.33
Qx3Z.fas       0.47      0.35      0.38
...
```

Include at least two decimal places in your output; do not add any quotes or other spurious characters. Do not worry if the columns do not all look aligned; as long as consecutive values on the same line are separated by a tab, that's fine. If some of the files in ```filelist``` do not exist, contain invalid data or otherwise cause an error, your function should ignore those files and continue. However, the names of those files should be returned by your function in a list. If all files are processed without errors your function should return an empty list.


In [26]:
# Your code here
#Write a function called AAtypetable 
#Argument is filelist
def AAtypetable(filelist, outputfile):
    # List for names of fractions for files
    fractions_list = []
    # List for names of problem files
    error_files = []
    
    for filename in filelist:
        try:
            
            sequence = readAAsequence(filename)
            
          
            fractions = AAtypes(sequence)
            fractions_list.append((filename, fractions))
        
        except Exception as e:
            error_files.append(filename)
    
    
    with open(outputfile, 'w') as out:
        out.write("# Filename\tPolar\tSmall\tHydro\n")
        for fname, (polar, small, hydro) in fractions_list:
            out.write(f"{fname}\t{polar:.2f}\t{small:.2f}\t{hydro:.2f}\n")
    
    return error_files

In [27]:
# Test code
import os
import os.path
for fname in ["_Ptest123.fas", "_Ptest456.fas"]:
    with open(fname, "wt") as _OUTF:
        _OUTF.write("> test\n")
        _OUTF.write("TESTTESTTESTTESTTEST\n")
_rv=AAtypetable(filelist=['_Ptest123.fas','_Ptest456.fas'], outputfile="_table_test789.txt")
_fe=os.path.isfile("_table_test789.txt")
assert _fe, "Cannot find output file - has it been created?"
os.remove("_table_test789.txt")
os.remove("_Ptest123.fas")
os.remove("_Ptest456.fas")
assert type(_rv) is type([]), "Function should return a list; it returns %r" % _rv
print("OK")

OK


# Question 4: Clustering proteins based on aminoacid usage

### Question 4.a: Computing the Euclidean distance

Write a function called ```distance``` that takes two lists of ```float``` of equal length as its arguments. The function should return the Euclidean distance between the two lists, considered as vectors in a space of dimensionality equal to the length of the lists. That is to say, it should compute the element by element difference between the two lists, square that, sum all the squares and take a square root of the final sum (see the equation [here](https://en.wikipedia.org/wiki/Euclidean_distance#Higher_dimensions)). Your function should raise a ```DimensionalityException``` if the two lists passed are not of the same length, or if they are both empty; otherwise it should return a ```float```.

In [34]:
# Run this cell to define the exception
class DimensionalityException(Exception):
    pass

In [44]:
# Your code here
#Will need to import math for 'sqrt'
import math
#Write a function called distance
#Function is taking two lists as arguments
def distance(list1, list2):
    
    if not list1 and not list2:
        raise DimensionalityException("The two lists are both empty.")
    if len(list1) != len(list2):
        raise DimensionalityException("The two lists must be of equal length.") #raise exception
    
    squared_differences = [(x - y) ** 2 for x, y in zip(list1, list2)]
    
    # Sum all the squares and then square root to get the Euclidean distance
    euclidean_distance = math.sqrt(sum(squared_differences))
    
    return euclidean_distance

In [45]:
# Test code
_dist=distance([1.0, 2.0], [1.5, -1.0])
assert type(_dist) is type(0.0), "Return value is not a float: %r" % _dist
print("OK")

OK


### Question 4.b: Reading data in tabular format

Write a function called ```readTable``` that takes a string called ```filename``` as its argument. The function should read from the file specified in the argument a TSV table formatted as described at the end of Question 3 (you do not necessarily have to answer Question 3 in order to do this; you can create one such table containing arbitrary values using an editor). The function should return a dictionary with the filenames as its keys; the value associated to each key should be a list of floats, containing in order the percentages of polar, small and hydrophobic AAs, as listed in the table. So for instance, if the argument file contains the table used as an example in Question 3, ```readTable``` should return a dictionary containing at least the following items: 
```
{ "Pxxx.fasta": [0.52, 0.22, 0.33],  "Qx3Z.fas": [0.47, 0.35, 0.38],
... }
```


In [37]:
# Your code here
#Write a function called readTable
def readTable(filename):
    
    table_data = {}

    with open(filename, 'r') as file:
        next(file)
        for line_number, line in enumerate(file, start=2):  # start=2 to account for the skipped header
            columns = line.strip().split('\t')
        
            if len(columns) != 4:
                raise ValueError(f"Line {line_number} has an incorrect number of columns: {line.strip()}")

            table_data[columns[0]] = [float(columns[1]), float(columns[2]), float(columns[3])]

    return table_data

In [38]:
# Test code
import os
with open("_testTable123.tsv", "wt") as OUTF:
    OUTF.write("# Filename\tPolar\tSmall\tHydro\n")
    OUTF.write("Pxxx.fas\t0.52\t0.22\t0.33\n")
    OUTF.write("Pyyy.fas\t0.47\t0.35\t0.38\n")
_tab=readTable(filename="_testTable123.tsv")
assert type(_tab) is type({}), "Return value is not a dict: %r" % _tab
assert ("Pxxx.fas" in _tab) and ("Pyyy.fas" in _tab), "Return dict missing keys"
os.remove("_testTable123.tsv")
print("OK")

OK


### Question 4.c: Computing a distance matrix

Write a function called ```distanceMatrix``` that takes as its argument two strings called ```inputfile``` and ```outputfile```, used to pass the names of the input and output file. The function should read from the input file a TSV table created as specified at the end of Question 3 (again, you do not necessarily have to answer Question 3 in order to do this; you can create a table with made-up data using an editor). Suppose data for N proteins are read. Your function should then create an output TSV file containing a matrix of size NxN (plus one row and one column for labels). The rows and columns of the matrix should correspond to individual protein file names, so that each matrix entry corresponds to a pair of proteins, say Pxxx and Pyyy. Such entry should contain the distance between the ```[polar, small, hydro]``` lists (considered as 3 dimensional vectors) corresponding to Pxxx and Pyyy. If done correctly, your matrix should be symmetric and have zeroes on the diagonal. Format the matrix with row and column labels as in the following 3-protein example (that contains random numbers and filenames):
```
# Filename      Pxxx.fas  Pyyy.fas  Pzzz.fas
Pxxx.fas        0.00      2.34      1.51
Pyyy.fas        2.34      0.00      1.82 
Pzzz.fas        1.51      1.82      0.00

```
The exact alignment is not important, as long as the data are separated by tabs and the first line starts with a hash (```#```). Likewise, the order in which the proteins are listed is not important, provided the same order is used along the rows and along the columns. Do not add any quotes or other spurious characters. Your function should not return any value.

In [42]:
# Your code here
# Write a function called distance Matrix
def distanceMatrix(inputfile, outputfile):
    
    data = readTable(inputfile)

    filenames = list(data.keys())

    matrix = {name: {other_name: 0.0 for other_name in filenames} for name in filenames}

    for i, name1 in enumerate(filenames):
        for j, name2 in enumerate(filenames):
            if i < j:  
                dist = distance(data[name1], data[name2])
                matrix[name1][name2] = dist
                matrix[name2][name1] = dist 

    with open(outputfile, 'w') as f:
        # Write header
        f.write('# Filename\t' + '\t'.join(filenames) + '\n')

        for name in filenames:
            f.write(name + '\t' + '\t'.join(f'{matrix[name][other_name]:.2f}' for other_name in filenames) + '\n')

In [43]:
# Test code
import os
import os.path
with open("_testTable123.tsv", "wt") as OUTF:
    OUTF.write("# Filename\tPolar\tSmall\tHydro\n")
    OUTF.write("Pxxx.fas\t0.52\t0.22\t0.33\n")
    OUTF.write("Pyyy.fas\t0.47\t0.35\t0.38\n")
_rv=distanceMatrix(inputfile="_testTable123.tsv", outputfile="_testTable123out.tsv")
_fe=os.path.isfile("_testTable123out.tsv")
assert _fe, "Cannot find output file - has it been created?"
with open("_testTable123out.tsv", "rt") as INF:
    _fl=INF.readline()
assert _fl[0]=='#', "First line of output file does not start with a #"
os.remove("_testTable123.tsv")
os.remove("_testTable123out.tsv")
assert type(_rv) is type(None),"Function should not return any value; it returns %r" % _rv
print("OK")

OK


### An application of this type of analysis

Now that you have this code running, try downloading a few FASTA sequences for the [antifreeze proteins](https://pdb101.rcsb.org/motm/120) of cold water fish. Then download the sequences of other families of related proteins (eg Hemoglobin, Insulin and P53) from other species. Process these sequences according to Questions 3 and 4. Have a look at the resulting distance matrix - is any of these groups of proteins characterised by a distinctive pattern of usage of aminoacids? NOTE: no marks are assigned for this section.

## Examiners

Dr Fabrizio Smeraldi *(f.smeraldi@qmul.ac.uk)*; Prof Conrad Bessant *(c.bessant@qmul.ac.uk)*